In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [47]:
def unpickle(file):
 '''Load byte data from file'''
 with open(file, 'rb') as f:
  data = pickle.load(f, encoding='latin-1')
  return data

In [48]:
def load_cifar10_data(data_dir):
 '''Return train_data, train_labels, test_data, test_labels
 The shape of data is 32 x 32 x3'''
 train_data = None
 train_labels = []

 for i in range(1, 6):
  data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
  if i == 1:
   train_data = data_dic['data']
  else:
   train_data = np.vstack((train_data, data_dic['data']))
  train_labels += data_dic['labels']

 test_data_dic = unpickle(data_dir + "/test_batch")
 test_data = test_data_dic['data']
 test_labels = test_data_dic['labels']

 train_data = train_data.reshape((len(train_data), 3, 32, 32))
 train_data = np.rollaxis(train_data, 1, 4)
 train_labels = np.array(train_labels)

 test_data = test_data.reshape((len(test_data), 3, 32, 32))
 test_data = np.rollaxis(test_data, 1, 4)
 test_labels = np.array(test_labels)


In [49]:
data_dir = 'cifar-10-batches-py'

In [51]:
train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

TypeError: cannot unpack non-iterable NoneType object

Train data:  (50000, 32, 32, 3)
Train filenames:  (50000,)
Train labels:  (50000,)
Test data:  (10000, 32, 32, 3)
Test filenames:  (10000,)
Test labels:  (10000,)
Label names:  (10,)


In [58]:

import numpy as np
import matplotlib.pyplot as plt
import pickle

"""
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 
training images and 10000 test images.
The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains 
exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random 
order, but some training batches may contain more images from one class than another. Between them, the training 
batches contain exactly 5000 images from each class.
"""


def unpickle(file):
    """load the cifar-10 data"""
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def load_cifar_10_data(data_dir, negatives=False):
    """
    Return train_data, train_filenames, train_labels, test_data, test_filenames, test_labels
    """

    # get the meta_data_dict
    # num_cases_per_batch: 1000
    # label_names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # num_vis: :3072

    meta_data_dict = unpickle(data_dir + "/batches.meta")
    cifar_label_names = meta_data_dict[b'label_names']
    cifar_label_names = np.array(cifar_label_names)

    # training data
    cifar_train_data = None
    cifar_train_filenames = []
    cifar_train_labels = []

    # cifar_train_data_dict
    # 'batch_label': 'training batch 5 of 5'
    # 'data': ndarray
    # 'filenames': list
    # 'labels': list

    for i in range(1, 6):
        cifar_train_data_dict = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            cifar_train_data = cifar_train_data_dict[b'data']
        else:
            cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
        cifar_train_filenames += cifar_train_data_dict[b'filenames']
        cifar_train_labels += cifar_train_data_dict[b'labels']

    cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
    if negatives:
        cifar_train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_train_data = np.rollaxis(cifar_train_data, 1, 4)
    cifar_train_filenames = np.array(cifar_train_filenames)
    cifar_train_labels = np.array(cifar_train_labels)

    # test data
    # cifar_test_data_dict
    # 'batch_label': 'testing batch 1 of 1'
    # 'data': ndarray
    # 'filenames': list
    # 'labels': list

    cifar_test_data_dict = unpickle(data_dir + "/test_batch")
    cifar_test_data = cifar_test_data_dict[b'data']
    cifar_test_filenames = cifar_test_data_dict[b'filenames']
    cifar_test_labels = cifar_test_data_dict[b'labels']

    cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
    if negatives:
        cifar_test_data = cifar_test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_test_data = np.rollaxis(cifar_test_data, 1, 4)
    cifar_test_filenames = np.array(cifar_test_filenames)
    cifar_test_labels = np.array(cifar_test_labels)

    return cifar_train_data, cifar_train_filenames, cifar_train_labels, \
           cifar_test_data, cifar_test_filenames, cifar_test_labels, cifar_label_names


if __name__ == "__main__":
    """show it works"""

    cifar_10_dir = 'cifar-10-batches-py'

    train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = \
        load_cifar_10_data(cifar_10_dir)

    print("Train data: ", train_data.shape)
    print("Train filenames: ", train_filenames.shape)
    print("Train labels: ", train_labels.shape)
    print("Test data: ", test_data.shape)
    print("Test filenames: ", test_filenames.shape)
    print("Test labels: ", test_labels.shape)
    print("Label names: ", label_names.shape)

    # Don't forget that the label_names and filesnames are in binary and need conversion if used.

    # display some random training images in a 25x25 grid
    num_plot = 5
    f, ax = plt.subplots(num_plot, num_plot)
    for m in range(num_plot):
        for n in range(num_plot):
            idx = np.random.randint(0, train_data.shape[0])
            ax[m, n].imshow(train_data[idx])
            ax[m, n].get_xaxis().set_visible(False)
            ax[m, n].get_yaxis().set_visible(False)
    f.subplots_adjust(hspace=0.1)
    f.subplots_adjust(wspace=0)
    plt.show()



In [62]:
train_data.shape

(50000, 32, 32, 3)

In [129]:
train_labels.shape

(50000, 10, 2)

In [64]:
test_data.shape

(10000, 32, 32, 3)

In [128]:
train_labels.shape

(50000, 10, 2)

In [104]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [133]:
check_image = train_data[0]
check_label = train_labels[0]
print(check.shape)
print(check_label)
train_data = train_data.reshape((50000, 32*32,3))
train_images = train_data.astype('float32') / 255
test_data = test_data.reshape((10000, 32*32,3))
test_images = test_data.astype('float32') / 255

(1024, 3)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [136]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(1024,3)))
network.add(layers.Dense(10, activation='softmax'))


In [142]:
network.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [143]:
#Test

In [207]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
img1 = load_img('train_images/shaheer (2).jpg')

In [202]:
#img1

In [175]:
x1 = img_to_array(img1)
#x = x.reshape((1,) + x.shape)

In [177]:
x1.shape

(4160, 3120, 3)

In [178]:
train_images=[]

In [179]:
train_images.append(x1)

In [184]:
img2 = load_img('train_images/shaheer (2).jpg')
x2 = img_to_array(img2)
train_images.append(x2)

In [185]:
img3 = load_img('train_images/shaheer (3).jpg')
x3 = img_to_array(img3)
train_images.append(x3)

In [189]:
img4 = load_img('train_images/shaheer (4).jpg')
x4 = img_to_array(img4)
train_images.append(x4)

In [191]:
img5 = load_img('train_images/shaheer (5).jpg')
x5 = img_to_array(img5)
train_images.append(x5)

In [192]:
train_images

[array([[[108.,  89.,  49.],
         [108.,  89.,  49.],
         [102.,  83.,  43.],
         ...,
         [157., 149.,  87.],
         [154., 146.,  84.],
         [161., 153.,  91.]],
 
        [[112.,  93.,  53.],
         [110.,  91.,  51.],
         [106.,  87.,  47.],
         ...,
         [159., 151.,  89.],
         [158., 150.,  88.],
         [156., 148.,  86.]],
 
        [[107.,  88.,  46.],
         [106.,  87.,  45.],
         [109.,  87.,  46.],
         ...,
         [156., 148.,  86.],
         [162., 154.,  92.],
         [156., 148.,  86.]],
 
        ...,
 
        [[103.,  83.,  58.],
         [102.,  82.,  57.],
         [102.,  82.,  57.],
         ...,
         [170., 180., 172.],
         [172., 182., 174.],
         [170., 180., 172.]],
 
        [[112.,  92.,  67.],
         [108.,  88.,  63.],
         [105.,  85.,  60.],
         ...,
         [169., 179., 171.],
         [170., 180., 172.],
         [167., 177., 169.]],
 
        [[106.,  84.,  60.],
 

In [197]:
import numpy as np
train_images = np.array(train_images)
train_images

array([[[[108.,  89.,  49.],
         [108.,  89.,  49.],
         [102.,  83.,  43.],
         ...,
         [157., 149.,  87.],
         [154., 146.,  84.],
         [161., 153.,  91.]],

        [[112.,  93.,  53.],
         [110.,  91.,  51.],
         [106.,  87.,  47.],
         ...,
         [159., 151.,  89.],
         [158., 150.,  88.],
         [156., 148.,  86.]],

        [[107.,  88.,  46.],
         [106.,  87.,  45.],
         [109.,  87.,  46.],
         ...,
         [156., 148.,  86.],
         [162., 154.,  92.],
         [156., 148.,  86.]],

        ...,

        [[103.,  83.,  58.],
         [102.,  82.,  57.],
         [102.,  82.,  57.],
         ...,
         [170., 180., 172.],
         [172., 182., 174.],
         [170., 180., 172.]],

        [[112.,  92.,  67.],
         [108.,  88.,  63.],
         [105.,  85.,  60.],
         ...,
         [169., 179., 171.],
         [170., 180., 172.],
         [167., 177., 169.]],

        [[106.,  84.,  60.],
       

In [198]:
train_images.shape


(6, 4160, 3120, 3)